# Dear Jupyter Notebook Reader

In [2]:
# import all necessary packages  
import pandas as pd 
import numpy as np
import os 
import seaborn as sns
from src import helpers
from src import Textprocessor as tp

In [6]:
train = helpers.read_df('data/train.csv', index_col='listing_id')
#test = pd.read_csv('data/test.csv', index_col='listing_id')
#reviews = pd.read_csv('data/reviews.csv', index_col='listing_id')

In [9]:
processor = Textprocessor.TextProcessor()

In [17]:
# missing values
train = helpers.change_data_types(train)
test = helpers.change_data_types(test)


In [20]:
reviews.head(5)

,reviewer_id,comments,review_id
listing_id,,,
8Z94Z3WMCO,93896,"The flat was bright, comfortable and clean and...",30672
8Z94Z3WMCO,97890,We stayed with Adriano and Valerio for a week ...,32236
8Z94Z3WMCO,104133,Adriano was a fantastic host. We felt very at ...,41044
8Z94Z3WMCO,122714,We had a most wonderful stay with Adriano and ...,48926
8Z94Z3WMCO,111543,I'm not sure which of us misunderstood the s...,58352
